In [27]:
import mapboxgl
from h3 import h3
import numpy as np
import pandas as pd
from geojson.feature import Feature, FeatureCollection

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

In [62]:
df = pd.read_csv('https://raw.githubusercontent.com/mapbox/mapboxgl-jupyter/master/examples/data/us-temperature.csv')

In [30]:
def encode_hexagon_str(lat, lon, resolution):
    return h3.geo_to_h3(lat, lon, res=resolution)

def hex_from_h3(h3_str):
    return {"type": "Polygon",
            "coordinates": [h3.h3_to_geo_boundary(h3_str, geo_json=True)]}

In [31]:
encode_hexagon_str_v = np.vectorize(encode_hexagon_str)

In [32]:
df['Temperature'] = df['Temperature'].astype(np.float64)
df['HexagonStr'] = encode_hexagon_str_v(df['Latitude'], df['Longitude'], 3)

In [42]:
df_hex = df[['Temperature', 'HexagonStr']].groupby(by='HexagonStr', as_index=False).mean().dropna()
df_hex['Hexagon'] = df_hex.apply(lambda x: hex_from_h3(x['HexagonStr']), axis=1)

In [43]:
features = []
for i, row in df_hex.iterrows():
    features.append(Feature(geometry=row['Hexagon'], properties={'Temperature': row['Temperature'], 
                                                                 'id': row['HexagonStr']}))

In [44]:
feature_collection = FeatureCollection(features)

In [55]:
color_stops_C = [
    [50.0,  '#2b0001'],
    [40.0,  '#6b1428'],
    [30.0,  '#b83667'],
    [25.0,  '#db6c55'],
    [20.0,  '#e19f43'],
    [15.0,  '#e1ce3a'],
    [10.0,  '#b7da42'],
    [5.0,   '#5bca4e'],
    [0.0,   '#4eb195'],
    [-5.0,  '#4176be'],
    [-10.0, '#5751ac'],
    [-15.0, '#2b1b69'],
    [-20.0, '#8d1c8d'],
    [-30.0, '#f4a5f3'],
    [-40.0, '#e3e3e3']
][::-1]

color_stops_F = [
    [120.0, '#2b0001'],
    [100.0, '#6b1428'],
    [90.0,  '#b83667'],
    [80.0,  '#db6c55'],
    [70.0,  '#e19f43'],
    [60.0,  '#e1ce3a'],
    [50.0,  '#b7da42'],
    [40.0,  '#5bca4e'],
    [30.0,  '#4eb195'],
    [20.0,  '#4176be'],
    [10.0,  '#5751ac'],
    [0.0,   '#2b1b69'],
    [-10.0, '#8d1c8d'],
    [-20.0, '#f4a5f3'],
    [-40.0, '#e3e3e3']
][::-1]

In [56]:
viz = mapboxgl.viz.ChoroplethViz(feature_collection,
                access_token=token,
                color_property='Temperature',
                color_stops=color_stops_F,
                color_function_type='interpolate',
                opacity=0.5,
                line_width=0.2,
                line_opacity=0.025,
                line_color='black',                 
                color_default='#00aeef',
                height='1080px',
                width='1920px',
                center=(-93, 38),
                zoom=4,
                add_snapshot_links=False
)

In [57]:
viz.create_html('map.html')